# Trade network analysis
**Brian Dew (brianwdew@gmail.com)**

`12_product_network.ipynb`

For `prod` product, calculate the eigenvector centralities, and shares of total trade by country. Save as csv in `data/prod`

METODO:


In [2]:
# LIBRARIES
import pandas as pd 
import numpy as np #for eigenvector calc
import networkx as nx 
import pickle #save graphs
%matplotlib inline

# FILEPATHS
path = r'C:/Working/trade_network/data/clean'
dloc = r'C:/Working/trade_network/data' #destination of final file
pickle_loc = r'C:/Working/trade_network/data/tmp'

Build network and complete network calculations

In [7]:
# Select product
prod = 271121  # product id to use

frame = []
# 
# 1. Build the network
#
    # Generate network G for the product above for all years 
for y in range(2008,2015):
    csv_file = path + '/baci07_' + str(y) + '_clean.csv'
    df  = pd.read_csv(csv_file, index_col='hs6', header=0)
    G   = nx.from_pandas_dataframe(df[df.index == prod], 'i', 'j', ['v', 'q'], nx.DiGraph())
#
# 2. Calculate indicators for the network
#
    # Calculate centrality measures
    outdeg_cent = nx.out_degree_centrality(G)
    indeg_cent =  nx.in_degree_centrality(G)
    outdeg = G.out_degree(weight='q')
    indeg = G.in_degree(weight='q')
    cim = nx.eigenvector_centrality_numpy(G, weight='q')
    cex = nx.eigenvector_centrality_numpy(G.reverse(), weight='q')
        
    # Set the indicators as node attributes
    indicators = {
        'c_od': outdeg_cent,   # eigenvector centrality of importers
        'c_id': indeg_cent,   # eigenvector centrality of exporters       
        'ecim': cim,   # eigenvector centrality of importers
        'ecex': cex,   # eigenvector centrality of exporters
        'outd': outdeg,# total exports for country
        'indg': indeg,  # total imports for country             
        
    }
    for k, v in indicators.iteritems():
        nx.set_node_attributes(G, k, v)
    
    # Temporarily store the network for that year
    pickle.dump(G, open(pickle_loc + '/graph_'+ str(y) + '_' + str(prod) + '.txt', 'w'))
    
#
# 3. Build dataframe and merge network, indicators, and imfdata
#
    df = pd.concat([pd.Series(j[1]) for j in indicators.iteritems()], axis=1, keys=[j[0] for j in indicators.iteritems()])
    df['year'] = pd.to_datetime(str(y)).year
    df['tot'] = G.size(weight='q') 
    df['shX'] = df['outd'] / df['tot']
    df['shM'] = df['indg'] / df['tot']
    df.index.names=['iso2']
    
    frame.append(df) ## store dataframes in list

combined = pd.concat(frame, axis=0)
combined.reset_index().set_index(['iso2','year']).to_csv('data/prod/final_'+ str(prod) + '.csv')

In [30]:
combined = pd.read_csv('data/prod/final_271111.csv', index_col=['iso2'])
combined[combined.index == 'JP']

,year,indg,outd,ecim,ecex,tot,shX,shM
iso2,,,,,,,,
JP,2008,6.926315e+07,54950.240,0.757817,1.147962e-11,1.593463e+08,3.448478e-04,0.434670
JP,2009,6.429959e+07,0.012,0.000301,8.389668e-17,1.637308e+08,7.329103e-11,0.392715
JP,2010,7.063423e+07,0.879,0.013567,1.165706e-08,2.169204e+08,4.052178e-09,0.325623
JP,2011,7.947143e+07,62087.810,0.162321,1.096995e-06,2.472070e+08,2.511572e-04,0.321477
JP,2012,8.700757e+07,1.434,0.939770,4.388614e-09,2.188700e+08,6.551834e-09,0.397531
JP,2013,9.166869e+07,1054.622,0.197507,5.290376e-11,2.549971e+08,4.135819e-06,0.359489
JP,2014,8.968102e+07,4.150,0.957923,3.039098e-11,2.514917e+08,1.650154e-08,0.356596
